In [1]:
import os
import joblib
import pandas as pd
import numpy as np
from scipy.sparse import load_npz

# --- 1. Adım: Veri Yollarını Tanımla ---
BASE_DIR = r"C:\Users\goktu\workspace\datacops"

# --- 2. Adım: RFE (SelectKBest) Çıktılarını Yükle ---
print("RFE (SelectKBest) ile seçilmiş 200 özellikli veriler yükleniyor...")
X_train_rfe = load_npz(os.path.join(BASE_DIR, "X_train_rfe.npz"))
X_test_rfe = load_npz(os.path.join(BASE_DIR, "X_test_rfe.npz"))

# RFE'den gelen orijinal indeksler ve isimler
rfe_indices = joblib.load(os.path.join(BASE_DIR, "rfe_feature_indices.pkl")) 
rfe_names = joblib.load(os.path.join(BASE_DIR, "rfe_feature_names.pkl")) 

print(f"X_train_rfe shape: {X_train_rfe.shape}")
print(f"X_test_rfe shape: {X_test_rfe.shape}")

# --- 3. Adım: Hedef Değişkenleri (y) Yükle ---
print("Hedef değişkenler (y_train, y_test) yükleniyor...")
df_train = pd.read_csv(os.path.join(BASE_DIR, "train_scaled.csv"))
df_test = pd.read_csv(os.path.join(BASE_DIR, "test_scaled.csv"))

target_col = "Crm Cd" 
y_train = df_train[target_col].astype(str)
y_test = df_test[target_col].astype(str)

print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

RFE (SelectKBest) ile seçilmiş 200 özellikli veriler yükleniyor...
X_train_rfe shape: (848024, 200)
X_test_rfe shape: (147207, 200)
Hedef değişkenler (y_train, y_test) yükleniyor...
y_train shape: (848024,)
y_test shape: (147207,)


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
import time

# ... (Hücre 1'den X_train_rfe ve y_train geldiğini varsayıyorum) ...

# --- 2. Adım: Hızlı eğitim için alt küme al ---
print("LASSO eğitimi için alt küme oluşturuluyor...")
X_lasso, _, y_lasso, _ = train_test_split(
    X_train_rfe,
    y_train,
    train_size=150000,
    stratify=y_train,
    random_state=42
)

print(f"LASSO alt küme boyutu:")
print(f"X_lasso: ", X_lasso.shape)
print(f"y_lasso: ", y_lasso.shape)


# --- 3. Adım: L1 cezalI Lojistik model (liblinear ile) ---
print("\nHızlı LASSO (LogisticRegression, L1, LIBLINEAR, OVR) model eğitimi başlıyor...")

start_time = time.time()

lasso = LogisticRegression(
    penalty='l1',
    solver='liblinear',
    C=0.1,
    multi_class='ovr',
    max_iter=5000, 
    random_state=42,
    tol=1e-3
)

# Modeli eğit
lasso.fit(X_lasso, y_lasso)

end_time = time.time()
print(f"Model eğitimi {end_time - start_time:.2f} saniye içinde tamamlandı.")

# --- 4. Adım: Sonuçları al ---
coefs = lasso.coef_
coef_abs = np.mean(np.abs(coefs), axis=0) 

# Sıfır olmayan katsayıları bul
idx_nonzero_within_rfe = np.where(coef_abs > 1e-6)[0] 

print(f"\nLASSO sonrası sıfır olmayan özellik sayısı: {len(idx_nonzero_within_rfe)}")
print(f"Örnek ilk 10 indeks (RFE uzayında): {idx_nonzero_within_rfe[:10]}")

LASSO eğitimi için alt küme oluşturuluyor...
LASSO alt küme boyutu:
X_lasso:  (150000, 200)
y_lasso:  (150000,)

Hızlı LASSO (LogisticRegression, L1, LIBLINEAR, OVR) model eğitimi başlıyor...


C:\Users\goktu\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Model eğitimi 269.86 saniye içinde tamamlandı.

LASSO sonrası sıfır olmayan özellik sayısı: 98
Örnek ilk 10 indeks (RFE uzayında): [ 0  1  2  3  4  5 17 18 19 47]


In [3]:
import joblib
from scipy.sparse import save_npz
import numpy as np

# --- 1. Adım: İsimleri ve Orijinal İndeksleri Hazırla ---
# RFE uzayındaki seçilmiş indeksleri, orijinal (tüm) uzaydaki indekslere çevir
idx_final = rfe_indices[idx_nonzero_within_rfe]

# Seçilen özelliklerin isimlerini al
feat_final = [rfe_names[i] for i in idx_nonzero_within_rfe]

print(f"Nihai feature sayısı: {len(idx_final)}")
print(f"İlk 20 özellik ismi: {feat_final[:20]}")

# --- 2. Adım: Tam Veri Setlerini Filtrele ---
print("\nTam matrisler seçilen özelliklere göre filtreleniyor...")
X_train_selected = X_train_rfe[:, idx_nonzero_within_rfe]
X_test_selected = X_test_rfe[:, idx_nonzero_within_rfe]

print(f"X_train_selected (yeni shape): {X_train_selected.shape}")
print(f"X_test_selected (yeni shape): {X_test_selected.shape}")

# --- 3. Adım: Yeni Matrisleri ve İsimleri Kaydet ---
print("Yeni veriler diske kaydediliyor...")

save_npz(os.path.join(BASE_DIR, "X_train_selected.npz"), X_train_selected)
save_npz(os.path.join(BASE_DIR, "X_test_selected.npz"), X_test_selected)

joblib.dump(idx_final, os.path.join(BASE_DIR, "selected_feature_indices.pkl"))
joblib.dump(feat_final, os.path.join(BASE_DIR, "selected_feature_names.pkl"))

print("\nLASSO seçilmiş özellikler BAŞARIYLA KAYDEDİLDİ ✅")

Nihai feature sayısı: 98
İlk 20 özellik ismi: ['DR_NO', 'TIME OCC', 'Rpt Dist No', 'Vict Age', 'LAT', 'LON', 'Date Rptd_2020-05-30', 'Date Rptd_2020-05-31', 'Date Rptd_2020-06-01', 'Date Rptd_2022-02-03', 'Date Rptd_2022-03-02', 'Date Rptd_2022-04-04', 'Date Rptd_2022-05-02', 'Date Rptd_2022-05-03', 'Date Rptd_2022-06-01', 'Date Rptd_2022-06-02', 'Date Rptd_2022-06-03', 'Date Rptd_2022-07-05', 'Date Rptd_2022-08-02', 'Date Rptd_2022-08-03']

Tam matrisler seçilen özelliklere göre filtreleniyor...
X_train_selected (yeni shape): (848024, 98)
X_test_selected (yeni shape): (147207, 98)
Yeni veriler diske kaydediliyor...

LASSO seçilmiş özellikler BAŞARIYLA KAYDEDİLDİ ✅
